# Memory-based Collaborative Filtering Recommendation


This notebook will showcase how a Recommender System is built with one type of Collaborative Filtering techniques, which models the users' interests by collecting other users' preferences(collaborating) to provide personalized recommendations(filtering). The Collaborative Filtering techniques are commonly devided into two types: Memory-based and Model-based. <br>
<br>
**Memory-based CF:**<br>
The rating matrix is directly used to calculate user/item similarity and make rating predictions. Finding nearest neighbors for predicting rating is the essence of this method.Two directions for the memory-based prediction are user-based similarity and item-based similarity. The former one predicts a rating by a user on a movie based on how other similar users predicted that movie, and the later one predicts the rating based on how other similar items are rated by that user. <br>
<br>
Here I visualized the similarity matrix calculation for both user-based and item-based approaches:

![similarity matrix](Similarity_Matrix_Calculation.png)
<br>
<br>
<br>
**Model-based CF:**<br>
The prediction relies on a offline learned model that is re-trained (updated) periodically. More information about this type of Collaborating Filtering will be provided in another notebook that showcase Matrix Factorization Recommender System building.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [193]:
# Load rating data
df1 = pd.read_csv('combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'])
df1.head()

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


How is the scale of the dimensions in this dataset?

In [194]:
# how many movies are there?
n_movies = df1.isnull().sum()[1]
print('# of movies: ',n_movies) # df.isnull().sum() will return the number oF missing values in each column.
# how many customers are there?
n_custs = df1[df1['Rating'].notnull()]['Cust_Id'].nunique()
print('# of customers: ', n_custs)
# How many ratings are there?
n_ratings = df1[df1['Rating'].notnull()].count()[1]
print('# of ratings: ', n_ratings) # df.count() will will return the number oF rows in each column.

# of movies:  4499
# of customers:  470758
# of ratings:  24053764


# Adjust data structure

In [196]:
def adjust_data(df1):
    df1_na = pd.DataFrame(pd.isnull(df1.Rating))
    df1_na = df1_na[df1_na['Rating'] == True]
    df1_na = df1_na.reset_index() # the index of original ratings that are NA will be in a separate column

    movieid_list = []
    movie_id = 1

    for i,j in zip(df1_na['index'][1:],df1_na['index'][:-1]):
        temp = np.full((1,i-j-1), movie_id)   # np.full(shape, the number you want to fill in those shape)
        movieid_list = np.append(movieid_list, temp)
        movie_id += 1

    # Add in the last record and corresponding length
    last_record = np.full((1,len(df1) - df1_na.iloc[-1, 0] - 1),movie_id)
    movieid_list = np.append(movieid_list, last_record)
    df1 = df1[df1['Rating'].notnull()]
    df1['Movie_Id'] = movieid_list.astype(int)
    df1['Cust_Id'] = df1['Cust_Id'].astype(int)
    return df1

df1 = adjust_data(df1)
print(df1.head())

C:\Users\olivi\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   Cust_Id  Rating  Movie_Id
1  1488844     3.0         1
2   822109     5.0         1
3   885013     4.0         1
4    30878     4.0         1
5   823519     3.0         1


C:\Users\olivi\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Remove unessential data for faster computing 

The rating distribution of movies and users are both very right skewed, meaning that most users have only rated a very small number of movies and a lot of movies have relatively lower number of users rated. Those less active users or less popular movies can be dropped for significant improve in compute, because their ratings are less likely to be informative.

What I am doing here:
1. Remove movies with too less ratings (less popular movies)
2. Remove customers with too less ratings (less active customers)

In [199]:

f = ['count','mean']

df_movie_summary = df1.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index
print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = df1.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index
print('Customer minimum times of review: {}'.format(cust_benchmark))


Movie minimum times of review: 1799.0
Customer minimum times of review: 52.0


In [200]:
df1 = df1[~df1['Movie_Id'].isin(drop_movie_list)]
df1 = df1[~df1['Cust_Id'].isin(drop_cust_list)]
print('New dimensions of the dataset: ',df1.shape)

New dimensions of the dataset:  (17337458, 3)


In [204]:
# map with movie titles
df_title =  pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])
df_title.head()

,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


# Collaborating Filtering Recommender System

UserId, MovieId, and Rating are the only 3 columns needed for collaborative filtering model.

In [202]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17337458 entries, 696 to 24056846
Data columns (total 3 columns):
Cust_Id     int32
Rating      float64
Movie_Id    int32
dtypes: float64(1), int32(2)
memory usage: 396.8 MB


## Data preprocessing

Due to the limited memory my laptop allows, I will use a 0.2% subset of the original data for modeling.

In [61]:
# Randomly sample 0.5% of the ratings dataset
df_subset = df1.sample(frac=0.002)
# Check the sample info
print(df_subset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34675 entries, 22240780 to 694150
Data columns (total 3 columns):
Cust_Id     34675 non-null int32
Rating      34675 non-null float64
Movie_Id    34675 non-null int32
dtypes: float64(1), int32(2)
memory usage: 812.7 KB
None


In [62]:
from sklearn import model_selection
train_data, test_data = model_selection.train_test_split(df_subset, test_size=0.2)
print(train_data.shape) 
print(test_data.shape)

(27740, 3)
(6935, 3)


Create user-item matrix for both training and test data respectively.

In [126]:
train_matrix = pd.pivot_table(train_data,values='Rating',index='Cust_Id',columns='Movie_Id')
test_matrix = pd.pivot_table(test_data,values='Rating',index='Cust_Id',columns='Movie_Id')


print(train_matrix.shape)
print(test_matrix.shape)

(24351, 1327)
(6701, 1101)


## Calculating Similarity Matrix 

There are a lot of different metrics to calculate the similarity such as Jaccard Similarity, Cosine Similarity, and Pearson Correlation. Particularly, Pearson Correlation is the best and safest way to calculate user-based similarity because it considered standardizing each user's ratings, in order to avoid situations like when the comparison is affected by different rating habits across users. But item-based similarity does not need the standardization because the ratings are all come from the same user. So item-based similarity matrix can be calculated by using either Cosine Similarity or Pearson Correlation.

I will provide two solutions that both utilize Pearson Similarity as the metric for user-based . Method 1 uses the built-in function, while method 2 calculates the same function step by step mannually.


- Calculating **user-based similarity matrix**
![user_based](pearson_correlation.PNG)

- Calculating **item-based similarity matrix**
![item_based](cosine_similarity.PNG)

### Method 1:

In [ ]:
user_similarity = train_matrix.corr(method = 'pearson')
item_similarity = train_matrix.T.corr(method = 'pearson')

### Method 2:

To avoid the impact of different rating style across users, I will normalize all the ratings of each user(user vector) before we calcuate the similarity.

In [129]:
def normalize(r):
    result = (r - r.mean())/(r.max() - r.min())
    return result

train_matrix_normalized = train_matrix.apply(normalize)
test_matrix_normalized = test_matrix.apply(normalize)
train_matrix_normalized = train_matrix_normalized.fillna(0)
test_matrix_normalized = test_matrix_normalized.fillna(0)

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(train_matrix_normalized)
item_similarity = cosine_similarity(train_matrix_normalized.T)

# output similarity matrix arrary 
print('user-based similarity matrix:')
print(user_similarity[:4,:4])
print('item-based similarity matrix:')
print(item_similarity[:4,:4])

user-based similarity matrix:
[[ 1.00000000e+00  3.97778111e-04  9.68736369e-05 -1.53646636e-03]
 [ 3.97778111e-04  1.00000000e+00  2.83623952e-03 -1.69078776e-03]
 [ 9.68736369e-05  2.83623952e-03  1.00000000e+00 -2.07810467e-03]
 [-1.53646636e-03 -1.69078776e-03 -2.07810467e-03  1.00000000e+00]]
item-based similarity matrix:
[[ 1.00000000e+00 -3.57839120e-04 -1.46643111e-04 -2.50134943e-04]
 [-3.57839120e-04  1.00000000e+00 -2.00435673e-04 -3.41891039e-04]
 [-1.46643111e-04 -2.00435673e-04  1.00000000e+00 -1.40107559e-04]
 [-2.50134943e-04 -3.41891039e-04 -1.40107559e-04  1.00000000e+00]]


In [ ]:
user_sim_df = pd.DataFrame(user_similarity, 
                           index = train_matrix_normalized.index, 
                           columns = train_matrix_normalized.index)
item_sim_df = pd.DataFrame(item_similarity, 
                           index = train_matrix_normalized.columns, 
                           columns = train_matrix_normalized.columns)

## Get similar users or items

Note: Please don't take the predicted results in this notebook seriously , because the collaborative filtering model is very bad in scalability. Due to the limit of memory in my laptop, I needed to downsample the to only 0.2% of the original data, which significantly caused the sparsity of the table. So I apologize for this and please know that no meanningful result is very normal and reasonable in this case.

In [190]:
def get_similar_users(userid, rating):
    similar_score = user_sim_df[userid]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending = False)
    
    return similar_score

print(get_similar_users(424,4)[:5])

Cust_Id
2453373    4.0
1381920    4.0
1589599    4.0
607560     4.0
618579     4.0
Name: 424, dtype: float64


In [191]:
def get_similar_movies(movie_id, rating):
    similar_score = item_sim_df[movie_id]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending = False)
    
    return similar_score

print(get_similar_movies(8,3)[:5])

Movie_Id
8       0.500000
3147    0.062437
256     0.036141
2164   -0.000071
774    -0.000071
Name: 8, dtype: float64


In [225]:
df_title[df_title['Movie_Id'].isin(get_similar_movies(8,3)[:10].index)]

,Movie_Id,Year,Name
7,8,2004.0,What the #$*! Do We Know!?
255,256,2000.0,Ghost Dog: The Way of the Samurai
585,586,1995.0,Shanghai Triad
773,774,2003.0,Foyle's War: Set 2
1178,1179,1997.0,The Education of Little Tree
1364,1365,1998.0,Kurt & Courtney
1997,1998,2005.0,Saving Face
2163,2164,2004.0,Rory O'Shea Was Here
2526,2527,2001.0,Quicksand
3146,3147,1964.0,The Twilight Zone: Vol. 25


## Predict the results


Because I have downsized the sample data to a relatively much smaller size and the extreme sparcity of this smaller data, it is very hard to be able to actually predict the results. So I will not run this code but leave it here for reference. 

The following function is using a common prediction function with a formula below:

- For prediction from **user-based similarity matrix**:
![formula](pred_user.PNG)

- For prediction from **item-based similarity matrix**:
![formula](pred_item.PNG)

In [184]:
# Predict the rating for a designated user on a designated item
# note: the 'topn' is the threshold that restricts the number of the similar users
def predict_rating(user, item, topn, type = 'user'):
    if type == 'user':
        avg_user_rating = train_matrix[user].mean()
        similar_users = user_sim_df[user_sim_df.index!= user][user].sort_values(ascending = False)[:topn]
        similar_userids = list(similar_users.index)
        denominator = 1
        for i in similar_users.values:
            denominator *= i 
        # retrieve the rating of each users on the movie 8
        numerator = 0
        for j in similar_userids:
            avg_rating = train_matrix.loc[j,:].mean()
            weight = similar_users[j]
            rating = train_matrix.loc[j,item]
            if rating != np.nan:
                numerator += weight*(rating-avg_rating)
            else:
                pass
        result = avg_user_rating + (numerator/denominator)
    elif type == 'item':
        similar_items = item_sim_df[item_sim_df.index!= item][item].sort_values(ascending = False)[:topn]
        similar_itemids = list(similar_items.index)
        denominator = 1
        for i in similar_users.values:
            denominator *= i 
        numerator = 0
        for j in similar_itemids:
            weight = similar_items[j]
            rating = trainmatrix.loc[user,j]
            if rating != np.nan:
                numerator += weight*rating
            else:
                pass
        result = (numerator/denominator)
    return round(result,2)

def predict(test_data, topn, type = 'user'):
    user_movies = test_data.as_matrix(columns = ['Cust_Id','Movie_Id'])
    pred_result = []
    if type = 'user':
        for row in user_movies:
            pred_result.append(predict_rating(row[0], row[1], topn, type = 'user'))
    elif type = 'user':
        for row in user_movies:
            pred_result.append(predict_rating(row[0], row[1], topn, type = 'item'))
    return pred_result

### Evaluation

There are many evaluation metrics but one of the most popular metric used to evaluate accuracy of predicted numerical values is Root Mean Squared Error (RMSE). I will use the mean_square_error (MSE) function from sklearn, where the RMSE is just the square root of MSE.

![rmse](rmse.PNG)

In [80]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [81]:
test_rating = np.array(test_data['Rating'])
# Predict ratings on the training data with both similarity score
user_prediction = predict(test_data, 50, type='user')
item_prediction = predict(test_data, 50, type='item')

# RMSE on the test data
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_rating)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_rating)))

User-based CF RMSE: 512635.12566851435
Item-based CF RMSE: 626677.1369669506


In [83]:
train_rating = np.array(train_data['Rating'])
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_rating)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_rating)))

User-based CF RMSE: 361634.0427684085
Item-based CF RMSE: 6627.045366193754


# Summary

From the implementation above, we can see some issues and challenges with memory-based Collaborating Filtering:

1. **Handling unknown users/items (Cold Start Problem)**
New users and movies have no history in the data, so this model is not able to predict the rating of those objects.<br>

2. **Data Sparsity**
The rating distribution of most users and movies are very right-skewed, which means there will be a huge sparsity in the rating matrix that affects the prediction accuracy. <br>

3. **Scalability**
The compute for similarity matrix is very expensive out of the nature huge matrix. <br>

4. **Dynamic Update**
The calculation purely depends on historical data, which contains ratings done in a wide range of period. So the result is hard to reflect the dynamic change of users' taste.